# Analyzing draft picks
### Author: Annie Staker
### Date: November 16, 2024
Goal: to identify relationships between draft pick acquisitions and team performance over the course of a few years.

Method: to create visualizations that encode the variables year, ranking, and draft pick to identify the most effective. 

### Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import math

### Load spreadsheets of data

In [2]:
teams_gen_df = pd.read_excel('./data/WNBA.xlsx', sheet_name='Teams - General')
teams_gen_df.head()

,Irrelevant #s,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,Year
0,1,New York Liberty,40,32,8,0.800,40.0,85.6,30.8,68.7,...,36.6,22.8,13.8,7.9,4.5,3.2,15.4,16.9,9.2,2024
1,2,Minnesota Lynx,40,30,10,0.750,40.4,82.0,30.1,67.3,...,34.3,23.0,14.3,8.6,4.2,3.7,16.4,15.7,6.4,2024
2,3,Connecticut Sun,40,28,12,0.700,40.3,80.1,29.3,65.9,...,33.5,19.9,13.3,8.2,3.7,4.6,16.1,18.1,6.5,2024
3,4,Las Vegas Aces,40,27,13,0.675,40.1,86.4,30.9,68.1,...,34.1,20.5,11.7,7.1,5.0,3.6,16.5,17.2,5.5,2024
4,5,Seattle Storm,40,25,15,0.625,40.3,83.2,31.1,71.3,...,34.7,20.7,13.1,9.3,5.2,4.6,16.5,16.4,4.5,2024


In [3]:
rankings_df = pd.read_excel('./data/WNBA.xlsx', sheet_name='Rankings and Roster')
rankings_df = rankings_df[['TEAM', 'Year', 'Ranking', 'Roster']]
rankings_df.head()

,TEAM,Year,Ranking,Roster
0,1. New York Liberty - x,2024,1.0,New York Liberty
1,2. Minnesota Lynx - x,2024,2.0,Minnesota Lynx
2,3. Connecticut Sun - x,2024,3.0,Connecticut Sun
3,4. Las Vegas Aces - x,2024,4.0,Las Vegas Aces
4,5. Seattle Storm - x,2024,5.0,Seattle Storm


In [4]:
draft_df = pd.read_excel('./data/WNBA.xlsx', sheet_name='Draft Data')
draft_df.tail()

,Player,Draft Year,Round,Round Number,Pick Number,Draft Team,College Team,Draft Info
1146,Vickie Johnson,1997,Elite Second,NaN,4,New York Liberty,NaN,"1997, Round , Pick 4"
1147,Janeth Arcain,1997,Elite Second,NaN,5,Houston Comets,NaN,"1997, Round , Pick 5"
1148,Mikiko Hagiwara,1997,Elite Second,NaN,6,Sacramento Monarchs,NaN,"1997, Round , Pick 6"
1149,Nancy Lieberman-Cline,1997,Elite Second,NaN,7,Phoenix Mercury,NaN,"1997, Round , Pick 7"
1150,Haixia Zheng,1997,Elite Second,NaN,8,Los Angeles Sparks,NaN,"1997, Round , Pick 8"


### Engineer the data

In [5]:
rankings_df['Roster'].unique()

array(['New York Liberty', 'Minnesota Lynx', 'Connecticut Sun',
       'Las Vegas Aces', 'Seattle Storm', 'Indiana Fever',
       'Phoenix Mercury', 'Atlanta Dream', 'Washington Mystics',
       'Chicago Sky', 'Dallas Wings', 'Los Angeles Sparks',
       'San Antonio Stars', 'Tulsa Shock', 'San Antonio Silver Stars'],
      dtype=object)

In [6]:
rankings_df[rankings_df['Roster'].isna()].head()

,TEAM,Year,Ranking,Roster


In [7]:
team_df = teams_gen_df.merge(rankings_df, left_on=['TEAM', 'Year'], right_on=['Roster', 'Year'])
team_df.head()

,Irrelevant #s,TEAM_x,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,STL,BLK,BLKA,PF,PFD,+/-,Year,TEAM_y,Ranking,Roster
0,1,New York Liberty,40,32,8,0.800,40.0,85.6,30.8,68.7,...,7.9,4.5,3.2,15.4,16.9,9.2,2024,1. New York Liberty - x,1.0,New York Liberty
1,2,Minnesota Lynx,40,30,10,0.750,40.4,82.0,30.1,67.3,...,8.6,4.2,3.7,16.4,15.7,6.4,2024,2. Minnesota Lynx - x,2.0,Minnesota Lynx
2,3,Connecticut Sun,40,28,12,0.700,40.3,80.1,29.3,65.9,...,8.2,3.7,4.6,16.1,18.1,6.5,2024,3. Connecticut Sun - x,3.0,Connecticut Sun
3,4,Las Vegas Aces,40,27,13,0.675,40.1,86.4,30.9,68.1,...,7.1,5.0,3.6,16.5,17.2,5.5,2024,4. Las Vegas Aces - x,4.0,Las Vegas Aces
4,5,Seattle Storm,40,25,15,0.625,40.3,83.2,31.1,71.3,...,9.3,5.2,4.6,16.5,16.4,4.5,2024,5. Seattle Storm - x,5.0,Seattle Storm


In [8]:
team_df.columns

Index(['Irrelevant #s',        'TEAM_x',            'GP',             'W',
                   'L',          'WIN%',           'MIN',           'PTS',
                 'FGM',           'FGA',           'FG%',        15:00:00,
                 '3PA',           '3P%',           'FTM',           'FTA',
                 'FT%',          'OREB',          'DREB',           'REB',
                 'AST',           'TOV',           'STL',           'BLK',
                'BLKA',            'PF',           'PFD',           '+/-',
                'Year',        'TEAM_y',       'Ranking',        'Roster'],
      dtype='object')

In [9]:
draft_df['Draft Team'].unique()

array(['Indiana Fever', 'Los Angeles Sparks', 'Chicago Sky',
       'Dallas Wings', 'Washington Mystics', 'Minnesota Lynx',
       'Connecticut Sun', 'New York Liberty', 'Atlanta Dream',
       'Seattle Storm', 'Las Vegas Aces', 'Phoenix Mercury',
       'Washington Mystics*', 'Atlanta Dream (via WAS)',
       'Washington Mystics (via ATL)', 'Indiana Fever (via LAS)',
       'Indiana Fever (via DAL)', 'Dallas Wings (via CHI)',
       'Las Vegas Aces (via PHX)', 'Los Angeles Sparks (via SEA)',
       'Indiana Fever (via MIN)', 'Las Vegas Aces (via IND)',
       'Atlanta Dream (via LAS)', 'Los Angeles Sparks (via WAS)',
       'Seattle Storm (via NYL)',
       'Seattle Storm (traded in principle to NYL)',
       'Los Angeles Sparks (via CHI)', 'Indiana Fever (via PHX)',
       'Phoenix Mercury (via ATL)', 'Dallas Wings (via WAS)',
       'New York Liberty (via CON)', 'Los Angeles Sparks (via DAL)',
       'Chicago Sky (via PHX)', 'Seattle Storm (traded to IND)',
       'Dallas Wings (via

In [10]:
draft_df[draft_df['Round Number'].isna()].head()

,Player,Draft Year,Round,Round Number,Pick Number,Draft Team,College Team,Draft Info
1135,Dena Head,1997,Elite First,NaN,1,Utah Starzz,NaN,"1997, Round , Pick 1"
1136,Isabelle Fijalkowski,1997,Elite First,NaN,2,Cleveland Rockers,NaN,"1997, Round , Pick 2"
1137,Rhonda Mapp,1997,Elite First,NaN,3,Charlotte Sting,NaN,"1997, Round , Pick 3"
1138,Kym Hampton,1997,Elite First,NaN,4,New York Liberty,NaN,"1997, Round , Pick 4"
1139,Wanda Guyton,1997,Elite First,NaN,5,Houston Comets,NaN,"1997, Round , Pick 5"


In [11]:
draft_df['Trimmed Draft Team'] = draft_df['Draft Team'].str.split('(', expand=False).str[0].str.strip()
draft_df.loc[draft_df['Player'] == 'Naomi Mullitauaopele', 'Draft Team'] = 'Utah Starzz'
draft_df.loc[draft_df['Draft Team'] == 'Chicago sky', 'Draft Team'] = 'Chicago Sky'
draft_df.loc[draft_df['Draft Team'] == 'Minnesota', 'Draft Team'] = 'Minnesota Lynx'
draft_df.loc[draft_df['Draft Team'] == 'Washington Mystics*', 'Draft Team'] = 'Washington Mystics'
draft_df['Round Number'] = draft_df['Round Number'].fillna(-1.0)
draft_df['Round Number'] = draft_df['Round Number'].astype(int)
draft_df.head()

,Player,Draft Year,Round,Round Number,Pick Number,Draft Team,College Team,Draft Info,Trimmed Draft Team
0,Caitlin Clark,2024,First,1,1,Indiana Fever,Iowa,"2024, Round 1, Pick 1",Indiana Fever
1,Cameron Brink,2024,First,1,2,Los Angeles Sparks,Stanford,"2024, Round 1, Pick 2",Los Angeles Sparks
2,Kamilla Cardoso,2024,First,1,3,Chicago Sky,South Carolina,"2024, Round 1, Pick 3",Chicago Sky
3,Rickea Jackson,2024,First,1,4,Los Angeles Sparks,Tennessee,"2024, Round 1, Pick 4",Los Angeles Sparks
4,Jacy Sheldon,2024,First,1,5,Dallas Wings,Ohio State,"2024, Round 1, Pick 5",Dallas Wings


In [12]:
draft_df['Round Number'].unique()

array([ 1,  2,  3,  4, -1])

In [13]:
draft_df['Trimmed Draft Team'].unique()

array(['Indiana Fever', 'Los Angeles Sparks', 'Chicago Sky',
       'Dallas Wings', 'Washington Mystics', 'Minnesota Lynx',
       'Connecticut Sun', 'New York Liberty', 'Atlanta Dream',
       'Seattle Storm', 'Las Vegas Aces', 'Phoenix Mercury',
       'Washington Mystics*', 'San Antonio Stars', 'Tulsa Shock',
       'Chicago sky', 'San Antonio Silver Stars', 'Sacramento Monarchs',
       'Detroit Shock', 'Houston Comets', 'Charlotte Sting',
       'Cleveland Rockers', 'Portland Fire', 'Minnesota', 'Utah Starzz',
       'Miami Sol', 'Orlando Miracle', 'Uath'], dtype=object)

In [14]:
team_year_df = draft_df.groupby(['Trimmed Draft Team', 'Draft Year']).agg(
    Number_of_Players_Drafted=('Player', 'count'),
    Number_of_First_Round_Drafts=('Round', lambda x: (x == 'First').sum())
).reset_index()

team_year_df.head(n=25)

,Trimmed Draft Team,Draft Year,Number_of_Players_Drafted,Number_of_First_Round_Drafts
0,Atlanta Dream,2008,3,1
1,Atlanta Dream,2009,3,1
2,Atlanta Dream,2010,3,1
3,Atlanta Dream,2011,3,1
4,Atlanta Dream,2012,2,0
5,Atlanta Dream,2013,2,0
6,Atlanta Dream,2014,3,1
7,Atlanta Dream,2015,3,1
8,Atlanta Dream,2016,4,1
9,Atlanta Dream,2017,3,1


In [15]:
team_rankings_df = team_year_df.merge(rankings_df, left_on=['Trimmed Draft Team', 'Draft Year'], right_on=['Roster', 'Year'])
team_rankings_df.head()

,Trimmed Draft Team,Draft Year,Number_of_Players_Drafted,Number_of_First_Round_Drafts,TEAM,Year,Ranking,Roster
0,Atlanta Dream,2010,3,1,4. Atlanta Dream - x,2010,4.0,Atlanta Dream
1,Atlanta Dream,2011,3,1,3. Atlanta Dream - x,2011,3.0,Atlanta Dream
2,Atlanta Dream,2012,2,0,3. Atlanta Dream - x,2012,3.0,Atlanta Dream
3,Atlanta Dream,2013,2,0,2. Atlanta Dream - x,2013,2.0,Atlanta Dream
4,Atlanta Dream,2014,3,1,1. Atlanta Dream - e,2014,1.0,Atlanta Dream


In [16]:
team_rankings_df.shape

(177, 8)

In [17]:
team_rankings_df = team_rankings_df.drop(columns=['TEAM', 'Roster', 'Year'])
team_rankings_df = team_rankings_df.rename(columns={'Trimmed Draft Team': 'Team', 'Number_of_Players_Drafted': 'Total Draft Picks', 'Number_of_First_Round_Drafts': 'Total First Round Draft Picks', 'Draft Year': 'Year'})
team_rankings_df.head()

,Team,Year,Total Draft Picks,Total First Round Draft Picks,Ranking
0,Atlanta Dream,2010,3,1,4.0
1,Atlanta Dream,2011,3,1,3.0
2,Atlanta Dream,2012,2,0,3.0
3,Atlanta Dream,2013,2,0,2.0
4,Atlanta Dream,2014,3,1,1.0


In [18]:
type(team_rankings_df['Ranking'][0])

numpy.float64

In [19]:
team_rankings_df[team_rankings_df['Ranking'].isna()].head()

,Team,Year,Total Draft Picks,Total First Round Draft Picks,Ranking
15,Chicago Sky,2010,2,1,NaN
29,Connecticut Sun,2010,4,2,NaN
90,Minnesota Lynx,2010,3,2,NaN
157,Tulsa Shock,2010,2,0,NaN


In [20]:
team_rankings_df.at[15, "Ranking"] = 12.0
team_rankings_df.at[29, "Ranking"] = 6.0
team_rankings_df.at[90, "Ranking"] = 3.0
team_rankings_df.at[157, "Ranking"] = 11.0
team_rankings_df[team_rankings_df['Ranking'].isna()].head()

,Team,Year,Total Draft Picks,Total First Round Draft Picks,Ranking


In [21]:
team_rankings_df['Ranking'] = team_rankings_df['Ranking'].astype(int)

In [22]:
team_rankings_df.head()

,Team,Year,Total Draft Picks,Total First Round Draft Picks,Ranking
0,Atlanta Dream,2010,3,1,4
1,Atlanta Dream,2011,3,1,3
2,Atlanta Dream,2012,2,0,3
3,Atlanta Dream,2013,2,0,2
4,Atlanta Dream,2014,3,1,1


### Create graphs
At this point we have 2 relevant dataframes:
- team_rankings_df: for each draft team each year, tells the total number of draft picks, total first round draft picks, and ranking.
- draft_df: for each drafted player each year, tells the draft team, college team, and draft year, round, and number.

In [23]:
team_rankings_df.head()

,Team,Year,Total Draft Picks,Total First Round Draft Picks,Ranking
0,Atlanta Dream,2010,3,1,4
1,Atlanta Dream,2011,3,1,3
2,Atlanta Dream,2012,2,0,3
3,Atlanta Dream,2013,2,0,2
4,Atlanta Dream,2014,3,1,1


In [24]:
draft_df.head()

,Player,Draft Year,Round,Round Number,Pick Number,Draft Team,College Team,Draft Info,Trimmed Draft Team
0,Caitlin Clark,2024,First,1,1,Indiana Fever,Iowa,"2024, Round 1, Pick 1",Indiana Fever
1,Cameron Brink,2024,First,1,2,Los Angeles Sparks,Stanford,"2024, Round 1, Pick 2",Los Angeles Sparks
2,Kamilla Cardoso,2024,First,1,3,Chicago Sky,South Carolina,"2024, Round 1, Pick 3",Chicago Sky
3,Rickea Jackson,2024,First,1,4,Los Angeles Sparks,Tennessee,"2024, Round 1, Pick 4",Los Angeles Sparks
4,Jacy Sheldon,2024,First,1,5,Dallas Wings,Ohio State,"2024, Round 1, Pick 5",Dallas Wings


Will generate graphs just for one team at a time.

In [25]:
team = 'Indiana Fever'
rankings_df_by_team = team_rankings_df[team_rankings_df['Team'] == team]

all_draft_picks = px.scatter(rankings_df_by_team, x='Total Draft Picks', y='Ranking', trendline='ols', title=f'Ranking over Total Draft Picks: {team}', hover_data=['Year', 'Ranking', 'Total Draft Picks'])
all_draft_picks.show()

first_round_draft_picks = px.scatter(rankings_df_by_team, x='Total First Round Draft Picks', y='Ranking', trendline='ols', title=f'Ranking over Total First Round Draft Picks: {team}', hover_data=['Year', 'Ranking', 'Total First Round Draft Picks'])
first_round_draft_picks.show()

In [26]:
team = 'Indiana Fever'
rankings_df_by_team = team_rankings_df[team_rankings_df['Team'] == team]

all_draft_picks = px.line(rankings_df_by_team, x='Year', y='Ranking', markers=True, title=f'Ranking by Year: {team}', hover_data=['Year', 'Ranking', 'Total Draft Picks'])
all_draft_picks.show()

In [27]:
team = 'Indiana Fever'
rankings_df_by_team = team_rankings_df[team_rankings_df['Team'] == team]

all_draft_picks = px.scatter(rankings_df_by_team, x='Year', y='Ranking', size='Total First Round Draft Picks', title=f'Ranking by Year, Size Encoding First Round Draft Picks: {team}', hover_data=['Year', 'Ranking', 'Total Draft Picks'], trendline="lowess", trendline_options=dict(frac=0.1))
all_draft_picks.show()

In [29]:
with pd.ExcelWriter('data/draft_picks.xlsx') as writer:
    team_rankings_df.to_excel(writer, sheet_name='Aggregated', index=False)
    draft_df.to_excel(writer, sheet_name='By Player', index=False)
